In [ ]:
# --------------------------------------------------------
# Install necessary Python packages for this notebook
# --------------------------------------------------------

!pip install pathway bokeh --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# --------------------------------------------------------
# Import essential Python libraries
# --------------------------------------------------------

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

import pathway as pw
import bokeh.plotting
import panel as pn


In [ ]:
# --------------------------------------------------------
# Load CSV and prepare for streaming into Pathway
# --------------------------------------------------------

# Read the original dataset
parking_raw_df = pd.read_csv('dataset.csv')

# Combine date and time into a single timestamp column
parking_raw_df['Timestamp'] = pd.to_datetime(
    parking_raw_df['LastUpdatedDate'] + ' ' + parking_raw_df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)

# Sort data by timestamp for consistency
parking_raw_df = parking_raw_df.sort_values('Timestamp').reset_index(drop=True)

# Save a minimal CSV for streaming into Pathway
parking_raw_df[['Timestamp', 'Occupancy', 'Capacity']].to_csv(
    'parking_stream.csv',
    index=False
)


In [ ]:
# --------------------------------------------------------
# Define the schema for reading data into Pathway
# --------------------------------------------------------

class ParkingLotSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int


In [ ]:
# --------------------------------------------------------
# Load the CSV into a streaming table using Pathway
# --------------------------------------------------------

# Replay the CSV as a simulated data stream
parking_table = pw.demo.replay_csv(
    'parking_stream.csv',
    schema=ParkingLotSchema,
    input_rate=1000
)


In [ ]:
# --------------------------------------------------------
# Define parameters for the dynamic pricing model
# --------------------------------------------------------

alpha_coefficient = 2.0      # sensitivity factor for occupancy effect
default_base_price = 10.0    # initial parking price in dollars
price_floor = 5.0            # minimum allowed price
price_ceiling = 20.0         # maximum allowed price


In [ ]:
# --------------------------------------------------------
# Parse timestamp strings into datetime objects
# --------------------------------------------------------

# Define the datetime format
timestamp_format = "%Y-%m-%d %H:%M:%S"

# Add a parsed datetime column to Pathway table
parking_table_time = parking_table.with_columns(
    timestamp_dt = parking_table.Timestamp.dt.strptime(timestamp_format)
)


In [ ]:
# --------------------------------------------------------
# Define the function to compute dynamic price
# --------------------------------------------------------

@pw.udf
def calculate_dynamic_price(occupancy, capacity):
    """
    Calculate dynamic parking price based on occupancy ratio.
    """
    # If capacity is zero (edge case), fallback to base price
    if capacity == 0:
        return default_base_price
    else:
        # Compute occupancy ratio
        occ_ratio = occupancy / capacity
        # Calculate price using linear model
        price = default_base_price + alpha_coefficient * occ_ratio

    # Clamp the price between floor and ceiling limits
    price = max(price_floor, min(price_ceiling, price))
    return price


In [ ]:
# --------------------------------------------------------
# Compute dynamic price for each record in the stream
# --------------------------------------------------------

parking_table_pricing = parking_table_time.with_columns(
    dynamic_price = calculate_dynamic_price(
        pw.this.Occupancy,
        pw.this.Capacity
    )
)


In [ ]:
# --------------------------------------------------------
# Visualize dynamic parking prices using Bokeh
# --------------------------------------------------------

pn.extension()

def plot_dynamic_price(stream_table):
    """
    Plot dynamic price evolution over time.
    """
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Dynamic Parking Price (Model 1)",
        x_axis_type="datetime"
    )
    fig.line(
        "timestamp_dt",
        "dynamic_price",
        source=stream_table,
        line_width=2,
        color="navy"
    )
    fig.scatter(
        "timestamp_dt",
        "dynamic_price",
        source=stream_table,
        size=6,
        color="red"
    )
    return fig

# Generate an interactive plot panel
price_visualization = parking_table_pricing.plot(
    plot_dynamic_price,
    sorting_col="timestamp_dt"
)

pn.Column(price_visualization).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
# --------------------------------------------------------
# Execute the Pathway data pipeline
# --------------------------------------------------------

%%capture --no-display
pw.run()


Output()